In [ ]:
!pip install transformers
!pip install -U datasets
!pip install torch
!pip install -U datasets fsspec huggingface_hub
!pip install pandas
!pip install nltk
!pip install sacrebleu
!pip install tqdm
!pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import load_dataset
import pandas as pd
import torch, textwrap, time, re
from nltk.tokenize import word_tokenize
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tqdm import tqdm
import numpy as np

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
)

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    torch_dtype="auto"
)

cot_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
dataset = "poetry_dataset.csv"
try:
    df_local_csv = pd.read_csv(dataset)
    print(f"Dataset CSV '{dataset}' caricato con successo!")
except FileNotFoundError:
    print(f"Errore: Il file '{dataset}' non è stato trovato. Assicurati che sia nella stessa directory del tuo script.")
except Exception as e:
    print(f"Si è verificato un errore durante il caricamento del file CSV: {e}")

#print(dataset)
df_original = pd.read_csv("poetry_dataset.csv")
new_df = df_original[['poem_zh', 'poem_en']].rename(columns={
    'poem_zh': 'source',
    'poem_en': 'target'
})

print("Nuovo DataFrame con 'source' e 'target':")
print(new_df)


Dataset CSV 'poetry_dataset.csv' caricato con successo!
Nuovo DataFrame con 'source' e 'target':
                                               source  \
0   君生我未生，我生君已老。\n君恨我生迟，我恨君生早。\n君生我未生，我生君已老。\n恨不生同...   
1   幾行歸塞盡，暮雨相呼失。\n渚雲低暗渡，未必逢矰繳。\n\n片影獨何之，寒塘欲下遲。\n關月...   
2   昨夜见军贴，可汗大点兵。\n军书十二卷，卷卷有爷名。\n阿爷无大儿，木兰无长兄，\n愿为市鞍...   
3                          床前明月光，疑是地上霜。\n举头望明月，低头思故乡。   
4                  无边落木萧萧下，不尽长江滚滚来。\n万里悲秋常作客，百年多病独登台。   
5   怒发冲冠，凭栏处、潇潇雨歇。\n抬望眼、仰天长啸，壮怀激烈。\n三十功名尘与土，八千里路云和...   
6                                        蛇本无足，\n添画余恨。   
7                                        守株待兔，\n幸复再得。   
8                                       亡羊补牢，\n 未为晚矣。   
9                                        愚公移山，\n志坚可成。   
10                                       井底之蛙，\n不知天地。   
11                     红豆生南国,\n春来发几枝。\n愿君多采撷,\n此物最相思。   
12                                       月光思君,\n梦里相逢。   
13                                       心悦君兮,\n此情不移。   
14                                       君心似我,\n

In [ ]:
SYSTEM_TRANSLATOR_MSG = textwrap.dedent("""
    You are an expert translator translating from Chinese to English.
    Your task is to translate the given Chinese poem into English
    Think step by step before providing the final translation.

    Instructions:
    - Translate the given Chinese poem into English.
    - Generate ONLY ONE output at a time.
    - If you are still reasoning or analyzing the sentence, write exactly:
      Thought: <your thought or analysis>
    - Provide the Thoughts only in English.
    - If you provide the same thought two times provide the final translation.
    - Provide the final translation in English.
    - When and only when the translation is complete, write exactly:
      FINAL: <translated sentence in English>

    Make sure the output always starts with "Thought:" or "FINAL:".
""")

In [ ]:
def one_step(messages, max_tokens=40, temperature=0.3):
    prompt = "".join(f"<|{m['role']}|>\n{m['content']}\n" for m in messages) + "<|assistant|>\n"

    device = "cuda" if torch.cuda.is_available() else "cpu"
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            max_new_tokens= max_tokens,
            temperature=0.5,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result[len(prompt):].strip().split("\n")[0]

In [ ]:
def chain_of_thought(user_query, max_steps=5, stream=False):
    messages = [
        {"role": "system", "content": SYSTEM_TRANSLATOR_MSG},
        {"role": "user", "content": user_query}
    ]
    scratchpad = []

    answer = "(nessuna risposta definitiva)"
    while True:
        assistant_line = one_step(messages)

        if stream:
            print(assistant_line)
            time.sleep(0.5)

        if assistant_line.startswith("FINAL:"):
            answer = assistant_line.removeprefix("FINAL:").strip()
            break


        if not assistant_line.startswith("Thought:"):
            assistant_line = "Thought: " + assistant_line

        scratchpad.append(assistant_line)
        messages.append({"role": "assistant", "content": assistant_line})

    return scratchpad, answer


In [ ]:
def judge_translation(reference, hypothesis):
    """
   Evaluate the quality of a hypothetical translation against a reference using the BLEU score.

    Args:
        reference (str): The reference sentence.
        hypothesis (str): The translated sentence to be evaluated.

    Returns:
        tuple: BLEU score (float) and qualitative assessment (str).
    """

    reference_tokens = [list(reference)]
    hypothesis_tokens = list(hypothesis)

    smoothie = SmoothingFunction().method1
    bleu_score = sentence_bleu(reference_tokens, hypothesis_tokens, smoothing_function=smoothie)

    if bleu_score > 0.75:
        judgment = "Ottimo"
    elif bleu_score > 0.5:
        judgment = "Buono"
    elif bleu_score > 0.25:
        judgment = "Mediocre"
    else:
        judgment = "Scarso"

    return bleu_score, judgment

In [ ]:
if __name__ == "__main__":
    for idx, row in new_df.iterrows():
        question = f"Translate the given Chinese poem into English: {row['source']}"

        print(f"\n=== ESEMPIO {idx+1} ===")
        thoughts, answer = chain_of_thought(question, stream=True, max_steps=3)



        print("TRADUZIONE FINALE:", answer)

        print("TRADUZIONE ATTESA :", row['target'])

        bleu, giudizio = judge_translation(row['target'], answer)
        print(f"BLEU Score: {bleu:.4f}")
        print("Giudizio del LLM Giudice:", giudizio)

    print("\n=== FINE ===")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



=== ESEMPIO 1 ===
Thought: The poem seems to be expressing a sense of regret and longing for a different timeline where both parties were born at the same time.
Thought: The first two lines are repeated, suggesting a theme or a significant moment in time.
Thought: The poem uses imagery of distance and separation to emphasize the longing.
Thought: The last lines introduce an image of a butterfly, which could symbolize transformation or change.
Thought: The overall tone of the poem is melancholic and reflective.
Thought: The title of the poem could provide context, but it is not given.
Thought: Based on the given lines, the best translation would be:
FINAL: When I was not born, you were; when I was, you were old.
TRADUZIONE FINALE: When I was not born, you were; when I was, you were old.
TRADUZIONE ATTESA : When you were born I was not yet born.
When I was born you were already mature.

What a pity for you that I was born late.
What a pity for me that you are now old.

When you were bor

KeyboardInterrupt: 

In [ ]:
def baseline_response(question, max_tokens=60):
    messages = [
        {"role": "system", "content": "You are a translator. Translate the following sentences directly from Chinese to English, without going through other languages, and do so as concisely as possible."},
        {"role": "user", "content": question}
    ]
    return one_step(messages, max_tokens=max_tokens)

In [ ]:
cot_bleu_scores = []
noncot_bleu_scores = []

if __name__ == "__main__":
    print("== INIZIO VALUTAZIONE ==")

    for idx, row in tqdm(new_df.iterrows(), total=len(new_df), desc="Valutazione esempi"):
        question = f"Translate the chinese poem into english: {row['source']}"
        expected = row['target']

        #Chain of Thought (CoT)
        thoughts, cot_answer = chain_of_thought(question, stream=False, max_steps=5)
        bleu_cot, _ = judge_translation(expected, cot_answer)
        cot_bleu_scores.append(bleu_cot)

        #Baseline
        noncot_answer = baseline_response(question)
        bleu_noncot, _ = judge_translation(expected, noncot_answer)
        noncot_bleu_scores.append(bleu_noncot)

    avg_cot_bleu = np.mean(cot_bleu_scores)
    avg_noncot_bleu = np.mean(noncot_bleu_scores)

    print("\n== RISULTATI FINALI ==")
    print(f"Media BLEU (CoT): {avg_cot_bleu:.4f}")
    print(f"Media BLEU (Non-CoT): {avg_noncot_bleu:.4f}")

    if avg_cot_bleu > avg_noncot_bleu:
        print("Il modello CoT ha tradotto meglio in media.")
    elif avg_noncot_bleu > avg_cot_bleu:
        print("Il modello Non-CoT ha tradotto meglio in media.")
    else:
        print("Le prestazioni dei due modelli sono equivalenti.")


== INIZIO VALUTAZIONE ==


Valutazione esempi: 100%|██████████| 26/26 [06:01<00:00, 13.92s/it]


== RISULTATI FINALI ==
Media BLEU (CoT): 0.3017
Media BLEU (Non-CoT): 0.2146
Il modello CoT ha tradotto meglio in media.
